## Temperature forecasting for different cities in the world

## Introduction
For this project you are asked to analyze three datasets, called respectively:
1. pollution_us_2000_2016.csv
2. greenhouse_gas_inventory_data_data.csv
3. GlobalLandTemperaturesByCity.csv

You are asked to extract from dataset 2 only the US countries (for which we have info in the other datasets) and to perform the following tasks:
- to measure how pollution and temperature create cluster tracing the high populated cities in the world
- to analyze the correlation between pollution data and temperature change.
- to predict the yearly temperature change of a given city over a given time period, using the <b>ARIMA model</b> for <b>time series forecasting</b>, that is a model for time series forecasting integrating AR models with Moving Average.
- (OPTIONAL) rank the 5 cities that will have a highest temperature change in US


### TASK1 :Cluster Analysis
You use K-means or DBSCAN to perform the cluster analysis, and create a new dataset where the cities are associated to the different identified clusters

### TASK 2: Correlation Analysis

You measure the correlation between:
- temperature and latitude
- temperature and pollution
- temperature change (difference between the average temperature measured over the last 3 years and the previous temperature) and pollution


### TASK 3: Predicting the Temperature of a Given City across a Specified Time Period
After reading the data in the temperature data set, for each city cluster, before applying the ARIMA model you perform the following steps:

- EDA
- data cleaning and preprocessing (Converting the 'dt' (date) column to DateTime format, removing NaN)
- feature selection
- make the time-series stationary
- check for stationarity : Calculating the Augmented Dickey-Fuller Test statistic 
- identify the (p, q) order of the ARIMA model using ACF partial autocorrelation plot

Then:

-fit the ARIMA model using the calculated p, q values.
-calculate the MSE with respect to the true temp. measurements to estimate the performance of the model


NOTE: ARIMA models need the data to be stationary i.e. the data must not exhibit trend and/or seasonality. To identify and remove trend and seasonality, we can use
- seasonal decomposition
- differencing

In [1]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA, ARMAResults
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets


import seaborn as sns

from gustavo_functions import *

%load_ext autoreload
%autoreload 2


## SECTION 1: Cluster Analysis

In [ ]:
# read the csv file containing the polluters 
# df_pollution = pd.read_csv#("cata/pollution_us_2000_2016.csv")

## SECTION 2: Correlation Analysis

##SECTION 3: ARIMA model for temperature forecasting

## Loading the data 

In [2]:
# read the csv file containing temperature data into a DataFrame
Global = pd.read_csv("data-project3/GlobalLandTemperaturesByCity.csv")

In [3]:
# read the csv file containing temperature data into a DataFrame
Green = pd.read_csv("data-project3/greenhouse_gas_inventory_data_data(1).csv")

In [4]:
Pollution = pd.read_csv("data-project3/pollution_us_2000_2016.csv")

## profiling the data 

In [ ]:
profile = ProfileReport(Global,title="GLTBC Profiling Report")

In [ ]:
profile2 = ProfileReport(Green,title="GGID2 Profiling Report")

In [38]:

profile3 = ProfileReport(Pollution,title="POLLUTION Profiling Report")

In [5]:
# profile.to_notebook_iframe()

In [6]:

# profile2.to_notebook_iframe()

In [50]:
#profile3.to_notebook_iframe()

In [ ]:
# To save the profile but in this case I dont'n want to save 
#profile.to_file("report.html")

## Cleaning the data

In [5]:
#Global.City.unique()
#print(list(Global.Country.unique()))
# matches = [country for country in Global.Country.unique() if "United States" in str(country)]
# print(matches)

Global.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [56]:
print(list(Global[Global.Country == "United States"].City.unique()))


['Abilene', 'Akron', 'Albuquerque', 'Alexandria', 'Allentown', 'Amarillo', 'Anaheim', 'Anchorage', 'Ann Arbor', 'Antioch', 'Arlington', 'Arvada', 'Atlanta', 'Aurora', 'Austin', 'Bakersfield', 'Baltimore', 'Baton Rouge', 'Beaumont', 'Bellevue', 'Berkeley', 'Birmingham', 'Boston', 'Bridgeport', 'Brownsville', 'Buffalo', 'Burbank', 'Cambridge', 'Cape Coral', 'Carrollton', 'Cary', 'Cedar Rapids', 'Chandler', 'Charleston', 'Charlotte', 'Chattanooga', 'Chesapeake', 'Chicago', 'Chula Vista', 'Cincinnati', 'Clarksville', 'Clearwater', 'Cleveland', 'Colorado Springs', 'Columbia', 'Columbus', 'Concord', 'Coral Springs', 'Corona', 'Corpus Christi', 'Costa Mesa', 'Dallas', 'Dayton', 'Denton', 'Denver', 'Des Moines', 'Detroit', 'Downey', 'Durham', 'East Los Angeles', 'Edison', 'El Monte', 'El Paso', 'Elizabeth', 'Escondido', 'Eugene', 'Evansville', 'Fairfield', 'Fayetteville', 'Flint', 'Fontana', 'Fort Collins', 'Fort Lauderdale', 'Fort Wayne', 'Fort Worth', 'Fremont', 'Fresno', 'Fullerton', 'Gaine

In [6]:
Green.head()
#print(list(Green.country_or_area.unique()))
# matches = [country for country in Green.country_or_area.unique() if "United States of America" in str(country)]
# print(matches)

,country_or_area,year,value,category
0,Australia,2014,393126.946994,carbon_dioxide_co2_emissions_without_land_use_...
1,Australia,2013,396913.936530,carbon_dioxide_co2_emissions_without_land_use_...
2,Australia,2012,406462.847704,carbon_dioxide_co2_emissions_without_land_use_...
3,Australia,2011,403705.528314,carbon_dioxide_co2_emissions_without_land_use_...
4,Australia,2010,406200.993184,carbon_dioxide_co2_emissions_without_land_use_...


In [58]:
Green[Green.country_or_area == "United States of America"].head()

,country_or_area,year,value,category
1049,United States of America,2014,5.556007e+06,carbon_dioxide_co2_emissions_without_land_use_...
1050,United States of America,2013,5.502551e+06,carbon_dioxide_co2_emissions_without_land_use_...
1051,United States of America,2012,5.349221e+06,carbon_dioxide_co2_emissions_without_land_use_...
1052,United States of America,2011,5.559508e+06,carbon_dioxide_co2_emissions_without_land_use_...
1053,United States of America,2010,5.688756e+06,carbon_dioxide_co2_emissions_without_land_use_...


In [ ]:
print(list(Global[Global.Country == "United States"].City.unique()))

In [57]:

print(list(Green[Green.country_or_area == "United States of America"].category.unique()))

['carbon_dioxide_co2_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent', 'greenhouse_gas_ghgs_emissions_including_indirect_co2_without_lulucf_in_kilotonne_co2_equivalent', 'greenhouse_gas_ghgs_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent', 'hydrofluorocarbons_hfcs_emissions_in_kilotonne_co2_equivalent', 'methane_ch4_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent', 'nitrogen_trifluoride_nf3_emissions_in_kilotonne_co2_equivalent', 'nitrous_oxide_n2o_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent', 'perfluorocarbons_pfcs_emissions_in_kilotonne_co2_equivalent', 'sulphur_hexafluoride_sf6_emissions_in_kilotonne_co2_equivalent', 'unspecified_mix_of_hydrofluorocarbons_hfcs_and_perfluorocarbons_pfcs_emissions_in_kilotonne_co2_equivalent']


In [49]:
#print(list(Pollution['County Code'].unique()))

print(list(Pollution['State'].unique()))

['Arizona', 'California', 'Colorado', 'District Of Columbia', 'Florida', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Michigan', 'Missouri', 'New Jersey', 'New York', 'North Carolina', 'Oklahoma', 'Pennsylvania', 'Texas', 'Virginia', 'Massachusetts', 'Nevada', 'New Hampshire', 'Tennessee', 'South Carolina', 'Connecticut', 'Iowa', 'Maine', 'Maryland', 'Wisconsin', 'Country Of Mexico', 'Arkansas', 'Oregon', 'Wyoming', 'North Dakota', 'Idaho', 'Ohio', 'Georgia', 'Delaware', 'Hawaii', 'Minnesota', 'New Mexico', 'Rhode Island', 'South Dakota', 'Utah', 'Alabama', 'Washington', 'Alaska']


In [53]:
#Pollution.head()
#print(list(Pollution['County Code'].unique()))
#Pollution.columns

#filtered_data = 
#Pollution[Pollution['County Code'] == 183].head()
#print(filtered_data)
#filtered_data.head(10)
Pollution.head()



,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [71]:
Pollution.shape

(1746661, 29)

## Cleaning the data 

In [7]:
# Load the datasets (make sure to update the file paths if necessary)
global_df = pd.read_csv("data-project3/GlobalLandTemperaturesByCity.csv")
green_df = pd.read_csv("data-project3/greenhouse_gas_inventory_data_data(1).csv")
pollution_df = pd.read_csv("data-project3/pollution_us_2000_2016.csv")

### 1 Cleaning the GlobalLandTemperaturesByCity dataset
# Filter only data for the United States
global_df = global_df[global_df["Country"] == "United States"].copy()

# Convert the date column to datetime format
global_df["dt"] = pd.to_datetime(global_df["dt"])

# Extract the year from the date column
global_df["Year"] = global_df["dt"].dt.year

# Remove NaN values in the temperature column
global_df.dropna(subset=["AverageTemperature"], inplace=True)

# Group by city and year, calculating the annual average temperature
global_clean = global_df.groupby(["City", "Year"])['AverageTemperature'].mean().reset_index()

# Extract unique latitude values per city
latitude_df = global_df[["City", "Latitude"]].drop_duplicates()

### 2 Cleaning the Greenhouse Gas dataset
# Filter only "United States of America"
green_df = green_df[green_df["country_or_area"] == "United States of America"].copy()

# Remove NaN values in the value column
green_df.dropna(subset=["value"], inplace=True)

# Keep only key columns
green_clean = green_df[["year", "value"]] #green_df[["year", "value", "category"]]

### 3 Cleaning the Pollution dataset
# Convert the date column to datetime format
pollution_df["Date Local"] = pd.to_datetime(pollution_df["Date Local"])

# Extract the year from the date column
pollution_df["Year"] = pollution_df["Date Local"].dt.year

# Select key columns
pollution_clean = pollution_df[["City", "Year", "NO2 Mean", "SO2 Mean", "CO Mean"]]

# Average pollution values by city and year
pollution_clean = pollution_clean.groupby(["City", "Year"]).mean().reset_index()

###  Merging the datasets
# Merge temperature and pollution data
merged_df = pd.merge(global_clean, pollution_clean, on=["City", "Year"], how="inner")

# Merge with greenhouse gas data
final_df = pd.merge(merged_df, green_clean, left_on="Year", right_on="year", how="inner")

# Drop the duplicate year column
final_df.drop(columns=["year"], inplace=True)

# Merge latitude data
final_df = pd.merge(final_df, latitude_df, on="City", how="left")

# Convert latitude values to numeric format
final_df["Latitude"] = final_df["Latitude"].str.replace("N", "").str.replace("S", "-").astype(float)

# Save the cleaned and merged dataset in CSV format
final_df.to_csv("cleaned_temperature_pollution_data.csv", index=False)

# Save the cleaned and merged dataset in Parquet format for efficient storage
#final_df.to_parquet("cleaned_temperature_pollution_data.parquet", index=False)

In [8]:
final_df.head()

,City,Year,AverageTemperature,NO2 Mean,SO2 Mean,CO Mean,value,Latitude
0,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,5.559508e+06,34.56
1,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,6.865398e+06,34.56
2,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,6.865398e+06,34.56
3,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,1.460806e+05,34.56
4,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,7.174237e+05,34.56


In [9]:
final_df.shape

(4750, 8)

In [10]:
final_df = final_df.rename(columns={'value': 'CO2-natural-pross'})

In [13]:
final_df.head()

,City,Year,AverageTemperature,NO2 Mean,SO2 Mean,CO Mean,CO2-natural-pross,Latitude
0,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,5.559508e+06,34.56
1,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,6.865398e+06,34.56
2,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,6.865398e+06,34.56
3,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,1.460806e+05,34.56
4,Albuquerque,2011,11.7585,13.406795,0.404034,0.205522,7.174237e+05,34.56


In [4]:
final_df = pd.read_csv("cleaned_temperature_pollution_data.csv")

In [5]:
print(list(final_df['category'].unique()))

['carbon_dioxide_co2_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent', 'greenhouse_gas_ghgs_emissions_including_indirect_co2_without_lulucf_in_kilotonne_co2_equivalent', 'greenhouse_gas_ghgs_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent', 'hydrofluorocarbons_hfcs_emissions_in_kilotonne_co2_equivalent', 'methane_ch4_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent', 'nitrogen_trifluoride_nf3_emissions_in_kilotonne_co2_equivalent', 'nitrous_oxide_n2o_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent', 'perfluorocarbons_pfcs_emissions_in_kilotonne_co2_equivalent', 'sulphur_hexafluoride_sf6_emissions_in_kilotonne_co2_equivalent', 'unspecified_mix_of_hydrofluorocarbons_hfcs_and_perfluorocarbons_pfcs_emissions_in_kilotonne_co2_equivalent']
